In [1]:
import sys
BIN = '../../'
sys.path.append(BIN)
import os.path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import time
import datetime

import torch
import torch.nn as nn
# import torch.optim as optim
import torch.utils.data

from torch.utils.data import TensorDataset

import my_matplotlib_style as ms

from fastai import basic_train, basic_data
from fastai.callbacks import ActivationStats
from fastai import train as tr

from my_nn_modules import AE_big, AE_3D_50, AE_3D_50_bn_drop, AE_3D_50cone, AE_3D_100, AE_3D_100_bn_drop, AE_3D_100cone_bn_drop, AE_3D_200, AE_3D_200_bn_drop, AE_3D_500cone_bn
from my_nn_modules import get_data, RMSELoss, plot_activations

In [2]:
path_to_data = '../../../data/split_data/'

save_dict = {}

module = AE_big
module_string = str(module).split("'")[1].split(".")[1]
model = module()


epochs = 20
lr = 1e-2
wd = 0
pp = 0
loss_func = nn.MSELoss()

bn_wd = False  # Don't use weight decay fpr batchnorm layers
true_wd = True  # wd will be used for all optimizers

In [4]:
ii = 0
# Load data
train = pd.read_pickle(path_to_data + 'sub_train_%d' % ii)
test = pd.read_pickle(path_to_data + 'sub_test_%d' % ii)

# Normalize
train_mean = train.mean()
train_std = train.std()

train = (train - train_mean) / train_std
test = (test - train_mean) / train_std

train_x = train
test_x = test
train_y = train_x  # y = x since we are training an AE
test_y = test_x

# Create DataBunch
bs = 1024
train_ds = TensorDataset(torch.tensor(train_x.values), torch.tensor(train_y.values))
valid_ds = TensorDataset(torch.tensor(test_x.values), torch.tensor(test_y.values))
train_dl, valid_dl = get_data(train_ds, valid_ds, bs=bs)
db = basic_data.DataBunch(train_dl, valid_dl)

In [12]:
learn = basic_train.Learner(data=db, model=model, loss_func=loss_func, bn_wd=bn_wd, true_wd=true_wd)
learn.load(module_string + '_subtrain_%d' % ii)
learn.model.eval()

AE_big(
  (en1): Linear(in_features=4, out_features=8, bias=True)
  (en2): Linear(in_features=8, out_features=6, bias=True)
  (en3): Linear(in_features=6, out_features=4, bias=True)
  (en4): Linear(in_features=4, out_features=3, bias=True)
  (de1): Linear(in_features=3, out_features=4, bias=True)
  (de2): Linear(in_features=4, out_features=6, bias=True)
  (de3): Linear(in_features=6, out_features=8, bias=True)
  (de4): Linear(in_features=8, out_features=4, bias=True)
  (tanh): Tanh()
)

In [9]:
learn.validate(dl=learn.data.valid_dl)

[0.0006899792]

In [10]:
learn.validate(dl=learn.data.train_dl)

[0.00068135734]

In [31]:
def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

def validate(learn, dl):
    for batch in dl:
        model = learn.model
        losses, nums = zip(*[loss_batch(model, loss_func, xb, yb) for xb, yb in dl])
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)
        print(val_loss)
        return val_loss

In [32]:
validate(learn, valid_dl)

0.0006899791656986553


0.0006899791656986553

In [33]:
validate(learn, train_dl)

0.0006813573956498598


0.0006813573956498598

In [ ]:
N = 10
for ii in np.arange(N):
    
    validate(learn, valid_dl)